## Glucocorticoid receptor - part 3 (rule aggregation)

### Import libraries

In [1]:
import pandas as pd

### Read in the datasets

In [2]:
df = pd.read_csv('nr3c1_agonists_stripped.csv')
df_pairs = pd.read_csv('nr3c1_agonists_pairs.csv')

### Filter pairs with MCS > 50% and radius >= 2

In [3]:
df_pairs = df_pairs[(df_pairs.percentmcs > 0.50) & (df_pairs.radius >= 2)]

### Determine which measurements can be included in delta property calculation

In [4]:
df['pchembl_exact'] = df.apply(lambda x: x.pchembl_value if x.standard_relation == "=" and x.pchembl_value else None, axis=1)

### Combine twice with confirmed molecular pairs

In [5]:
df_merge = df_pairs.merge(df, left_on='smiles1', right_on='stripped_smiles')
df_merge = df_merge.merge(df, left_on=['smiles2', 'target_pref_name', 'standard_type'], right_on=['stripped_smiles', 'target_pref_name', 'standard_type'])
df_merge['pchembl_exact_delta'] = df_merge['pchembl_exact_y'] - df_merge['pchembl_exact_x'] 
df_merge.sample(3).transpose()

,4460,2186,12220
smiles1,C[C@]12C[C@H](O)[C@H]3[C@@H](CCC4=CC(=O)C=C[C@...,Cn1c(SCC(=O)[C@@]2(O)CC[C@H]3[C@@H]4CCC5=CC(=O...,CC1CC2C(C(O)CC3(C)C2CCC3(O)C(=O)CO)C2(C)C=CC(=...
smiles2,C[C@]12C[C@H](O)[C@H]3[C@@H](CCC4=CC(=O)C=C[C@...,C[C@]12C[C@H](O)[C@H]3[C@@H](CCC4=CC(=O)C=C[C@...,C[C@]12C=CC(=O)C=C1CC[C@@H]1[C@H]3C[C@@H](O)[C...
percentmcs,0.853659,0.725,0.53125
radius,2,2,4
valid,True,True,True
...,...,...,...
ligand_efficiency.le_y,0.33,0.44,NaN
stripped_smiles_y,C[C@]12C[C@H](O)[C@H]3[C@@H](CCC4=CC(=O)C=C[C@...,C[C@]12C[C@H](O)[C@H]3[C@@H](CCC4=CC(=O)C=C[C@...,C[C@]12C=CC(=O)C=C1CC[C@@H]1[C@H]3C[C@@H](O)[C...
achiral_smiles_y,CC12C=CC(=O)C=C1CCC1C2C(O)CC2(C)C1CCC2(O)C(=O)...,CC12C=CC(=O)C=C1CCC1C2C(O)CC2(C)C1CCC2(O)C(=O)CO,CC12C=CC(=O)C=C1CCC1C3CC(O)C(O)(C(=O)CO)C3(C)C...
pchembl_exact_y,8.42,8.39,7.7


### Aggregate data by the two fragments

In [6]:
indexcols = ['fragment1', 'fragment2', 'radius', 'target_pref_name', 'standard_type'] #, 'assay_chembl_id', 'standard_units']
df_agg_data = pd.pivot_table(df_merge, values=['pchembl_exact_delta', 'percentmcs'], index=indexcols, aggfunc=['count','mean'])
df_agg_data.columns = ['_'.join(col).strip() for col in df_agg_data.columns.values]

### Retain single SMIRKS per transformation (effectively at random)

In [7]:
df_agg_smirks = pd.pivot_table(df_merge, values='smirks', index=indexcols, aggfunc='first')
df_agg = df_agg_data.join(df_agg_smirks).reset_index()

### Write to file

In [8]:
df_agg.to_csv('nr3c1_agonists_transformations.csv', index=False)